# IMEC2201 Herramientas Computacionales 
## Taller Semana 2: Visualizaciones y Ajuste de Datos
### Fecha Límite Entrega (Bloque Neón): <font color="#FF5733">23.59h de Abril 22, 2022</font>

Universidad de los Andes — Abril 6, 2022.

___
### Tener en Cuenta
La entrega del Taller 2 se puede realizar de tres maneras:
1. Cargar la solución en su repositorio en GitHub y enviar el enlace del mismo vía Bloque Neón.
2. Enviar vía Bloque Neón el archivo en Jupyter Notebook (extensión '.ipynb').
3. Enviar vía Bloque Neón el archivo PDF del Jupyter Notebook.
___

## Ejercicio 1: Correlación Medición-Simulación

En la carpeta $\texttt{data}$ se disponen los archivos `SD_5Min.xlsx` y `solar_production.csv`.

- `SD_5Min.xlsx` contiene las **mediciones** de los valores de producción de la planta solar fotovoltaica ubicada en el edificio Santo Domingo de la Universidad de los Andes, desde 1-Agosto-2019 hasta 30-Junio-2021.
- `solar_production.csv` contiene los resultados de una **simulación** de la misma planta y con la misma resolución de los datos (i.e., un dato cada cinco minutos), desde 1-Enero-2020 hasta 1-Enero-2021.

<div class="alert alert-block alert-warning">
    
Realice una correlación entre los datos medidos (eje X) y los datos simulados (eje Y) de Potencia AC. Tenga en cuenta:
1. ¿Cómo uniría los dos DataFrame en un único DataFrame con base en las fechas indicadas en el archivo `solar_production.csv`? Explore la función `innerjoin()`.
2. Cómo realizaría un ajuste lineal de los datos para: <br>
    2.1. Extraer parámetros de ecuación de la curva de regresión lineal (pendiente e intercepto). <br>
    2.2. Calcular el coeficiente de correlación R2. <br>
3. Grafique la correlación entre estos datos y su ajuste lineal, incluyendo en la leyenda la ecuación de la recta y el coeficiente de correlación.

**Nota:** Es posible experimentar retardos (lags) en VS Code o Jupyter Notebook por la cantidad de datos con que se trabaja y la alta resolución de las gráficas. Para evitar esto:
- Tomar el número de datos que ustedes desean graficar, ya sea modificando el DataFrame desde Julia o modificar el archivo CSV y guardarlo como un archivo aparte del original.
- Cambiar el backend de gráficas de Julia. Por ejemplo, para los que conocen matplotlib en Python, desde Julia pueden crear estas gráficas con la librería PyPlot.

</div>

___

**Ejemplo** <br>
<font color="#FF5733">Output</font>

<img src='./img/Correlation_Pac.png' width='350'/>

___

In [1]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()
Pkg.add("XLSX")
Pkg.add("CSV")
Pkg.add("DataFrames")

  Activating project at `c:\Users\danie\OneDrive\Documentos\GitHub\ua-imec2001-hc-202210\Herrmainetas C\content\week2`
    Updating registry at `C:\Users\danie\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\danie\OneDrive\Documentos\GitHub\ua-imec2001-hc-202210\Herrmainetas C\content\week2\Project.toml`
  No Changes to `C:\Users\danie\OneDrive\Documentos\GitHub\ua-imec2001-hc-202210\Herrmainetas C\content\week2\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\danie\OneDrive\Documentos\GitHub\ua-imec2001-hc-202210\Herrmainetas C\content\week2\Project.toml`
  No Changes to `C:\Users\danie\OneDrive\Documentos\GitHub\ua-imec2001-hc-202210\Herrmainetas C\content\week2\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\danie\OneDrive\Documentos\GitHub\ua-imec2001-hc-202210\Herrmainetas C\content\week2\Project.toml`
  No Changes to `C:\Users\danie\OneDrive\Documentos\GitHub\ua-imec2001-hc-202210\Herrma

In [ ]:
using DataFrames, XLSX
using DataFrames, CSV
df = DataFrame(XLSX.readtable("./data/SD_5Min(1).xlsx", "Radiacion", header=true)...)
#CSV.File("./data/solar_production.csv", header=1, delim=';')
df2 = DataFrame(CSV.File("./data/solar_production.csv", header=1, delim=';'))

In [ ]:
using Dates
fecha = []
df2[!,1][2][1:19]
for i in 1:size(df2[!,1])[1]
    push!(fecha,Dates.DateTime(df2[!,1][i][1:19],"y-m-d H:M:S"))
end
df2[:,"Date"]= fecha
df2 = df2[:,["Date","Zenith (deg)","Elevation (deg)","Azimuth (deg)","Airmass Relative (ad)","Airmass Absolute (ad)","Extraterrestrial Radiation (W/m2)","POA (W/m2)","Tmod (C)","Pdc (W)","Pac (W)","Daily Energy (Wh)","Weekly Energy (Wh)","Monthly Energy (Wh)"]]


In [ ]:
using DataFrames
df3=innerjoin(df,df2,on="Date")

In [39]:
using Plots
plot(df3."Meteocontrol Irrad (W/m2)",df3."POA (W/m2)",500)
keys

In [ ]:
function p1()
    if df2 ∉ 
end

## Ejercicio 2: Comportamiento Estadístico Mensual

En la carpeta $\texttt{data}$ se dispone el archivo `SD_5Min.xlsx` que contiene las **mediciones** de los valores de producción de la planta solar fotovoltaica ubicada en el edificio Santo Domingo de la Universidad de los Andes, desde 1-Agosto-2019 hasta 30-Junio-2021.

<div class="alert alert-block alert-warning">
    
Escriba un algoritmo que permita obtener los valores máximo, promedio y mínimo para todos los días de un mes durante cada instante de tiempo. Es decir:

<img src='./img/table_excercise2.png' width='650'/>

Note que:
- La primera columna contiene las estampas de tiempo para un día completo, es decir, un dato cinco minutal desde las 00:00 hasta las 23:55.
- Se tiene una columna de datos de irradiancia por día; es decir, 30 o 31 columnas de datos.
- Se agregan las columnas de interés: mínimo, promedio y máximo. Estas son columnas calculadas con las 30 o 31 columnas de datos. Por ejemplo:
    - Columna: Mínimo, Fila: 11:55 -> min(800, 700, 880, ..., 630) = 630
    - Columna: Promedio, Fila: 12:00 -> mean(850, 680, 930, ..., 720) = 795
    - Columna: Máximo, Fila: 12:05 -> max(1010, 810, 850, ..., 760) = 1010
- La gráfica se genera al graficar en una misma figura: (i.) Tiempo vs Mín, (ii.) Tiempo vs Prom, (iii.) Tiempo vs. Max.
</div>

___

**Ejemplo** <br>
<font color="#FF5733">Output</font>
<img src='./img/irradiance_per_minute.png' width='750'/>

**Nota:** Solo se solicita la gráfica de los datos mínimo, promedio y máximo, es decir, los marcadores color azul, verde y naranja en esta figura.
___

## Ejercicio 3: Histogramas y Distribuciones

En la carpeta $\texttt{data}$ se dispone el archivo `SD_5Min.xlsx` que contiene las **mediciones** de los valores de producción de la planta solar fotovoltaica ubicada en el edificio Santo Domingo de la Universidad de los Andes, desde 1-Agosto-2019 hasta 30-Junio-2021. 

<div class="alert alert-block alert-warning">
    
Similar al Ejercicio 2, escriba un algoritmo que permita obtener un DataFrame tal que:
- La primera columna contiene las estampas de tiempo para un día completo, es decir, un dato cinco minutal desde las 00:00 hasta las 23:55.
- Se tiene una columna de datos de irradiancia por día; es decir, 30 o 31 columnas de datos.

A partir de este DataFrame, cree un histogramas para tres instantes de tiempo diferentes (es decir, tres histogramas en total) y analice e interprete el resultado.

**Nota:** Explore qué es un *QQ-Plot* y cómo implementarlo en Julia.

</div>

___
**Ejemplo** <br>
<font color="#FF5733">Output</font>

<img src='./img/histograms.png' width='700'/>
___